In [ ]:
## SoftAE_ClassTest Initialization: ALWAYS RUN FIRST AFTER KERNEL RESTART
# 
# created: 05 nov 2024 by pavel
# Trying to import our SoftAE classes separately and run them in an integrated fashion.

import pyvisa
from matplotlib import pyplot as plt
from SoftAE_classPkg import Keithley_class as mm
from SoftAE_classPkg import stage_class as st
from SoftAE_classPkg import tempControl_class as t
from SoftAE_classPkg import ThorLabsCamera_class as cam
from SoftAE_classPkg import syringe_class as syr

## This next section is for the ESPico code to orient itself in directory space 
# when using an ipynb. Maybe there's a way to shove this into the class definition file.

%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('./SoftAE_classPkg')) # specify the non-workspace directory containing the source code
sys.path.insert(0, module_path)

## Now these ESPico things will import in peace:
import SoftAE_classPkg.ESPico_class as EP
import SoftAE_classPkg.SoftAE_ESPico_mscr_library as mscr

rm = pyvisa.ResourceManager()

#a = mm.Keithley('USB0::0x05E6::0x6510::04433485::INSTR',rm
b = st.stage('ASRL7::INSTR', 921600)
temp = t.tempControl("stage","com6",115200,0,1) #Novus instance
d = syr.syringe('ASRL4::INSTR', 115200, 14.4)
lamp = cam.Lamp("Lamp") #POM lamp

In [ ]:
# Template: Stand-alone POM Snapshot
import time

lamp = cam.Lamp("Lamp1")
cam1 = cam.Camera("cam1")
# Start temp control
# temp.get_sp()
# temp.get_pv()
#temp.write_sp(20)

# Start ThorLabs camera acquisition

lamp.on()

cam1.open()
arr = cam1.acquire_n_frames(1,0.1) #1 frame with 0.5 s exposure
cam1.close()
plt.imshow(arr)
plt.show()

lamp.off()



In [ ]:
# Template: Stage heating control
import pyvisa
from SoftAE_classPkg import tempControl_class as t
temp = t.tempControl("stage","com6",115200,0,1) #Novus instance
print(temp.get_sp())
print(temp.get_pv())
print(temp.get_pv_surf())
#temp.write_sp(52)

In [ ]:
# Template: Stand-alone EIS Measurement template 
import time
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('./SoftAE_classPkg')) # specify the non-workspace directory containing the source code
sys.path.insert(0, module_path)

## Now these ESPico things will import in peace:
import SoftAE_classPkg.ESPico_class as EP
import SoftAE_classPkg.SoftAE_ESPico_mscr_library as mscr

port = EP.palmsens.serial.auto_detect_port() # find the ESPico instrument (only one attached)
p = EP.ESPico('Pico1',port) # name and specify the port of this instance of ESPico class.

chlist = [14] # Define multiplexer channel list
# if you want to select all channels within a range use the below: 

#for i in range(1,6): #range is exclusive of end index: range(m,n) is m to n-1
#    chlist.append(i)

plotlist = [14] #if you want to selectively see the data

scriptPath = 'SoftAE_classPkg/scripts/testing.mscr'
outPath = 'SoftAE_classPkg/output'

# multichannel data recall and selective plotting
for ch in chlist:
    mscr.eis_run_mscrbuild(scriptPath,mux_ch=ch,mVac=10,20000,100,50,0) # write custom EIS MethodScript
    #mscr.lsv_run_mscrbuild(scriptPath,ch) # write custom LSV MethodScript
    c = p.sendscript_getdata(scriptPath,outPath,ch) #send script to run, c is output data
    if ch in plotlist:
        #p.lsv_dataplot(c,ch)
        data = p.eis_dataplot(c,ch)
        print(f'High-freq Z: {str(data[1][0])} Ohms')
        print(f'Lo-freq Z: {str(data[1][-1])} Ohms')


In [ ]:
b.move_to(45,50)

In [ ]:
# Template: EmStat Pico-measured data retrieval from directory
from glob import glob

#choose directory
target_dir = r"C:\Users\Osuji\Documents\Users\Pavel\2025_0122_LiqHandling_PEDOT_tests\PEDOT_EIS"

os.chdir(target_dir)
filelist = glob("*.txt")

for file in filelist:
    bop = EP.ESPico.eis_datafromfile(target_dir+'/'+f"{file}")
    print(bop[1][0]) #high-frequency impedance in Ohms
    print(bop[1][-1]) #low-freq Z, Ohms

In [10]:
# Current stage coordinates setup (could vary any given day) (Microscope, Dispenser, Flush, Wicking station positioning, absolute/relative coords)

flush_pos = [-31.5,14]
wick_pos = [-20,50]
dep_1 = [45,50] #Channel 1 electrode deposition position
ins_offset_2x = [-49,-39] #Relative inspection-deposition offset for 2x objective
ins_1 = [-4,11] #first electrode inspection position

#Below: list of all dropcasting locations for EIS_IDE_v1 PCB electrode array relative to dep_1
rel_dep_list = ([0,0,0,0,-15,-15,-15,-15,-30,-30,-30,-30,-45,-45,-45,-45],[0,-15,-30,-45,0,-15,-30,-45,0,-15,-30,-45,0,-15,-30,-45])


In [ ]:
import time

b.move_to(45,35)
d.single_pump(10000,0,300,20)
time.sleep(30)
d.head_descend()
d.single_pump(10000,0,300,20)
time.sleep(30)
d.head_retract()
b.move_to(-20,50)
time.sleep(5)
d.head_descend()
time.sleep(5)
d.head_retract()

In [ ]:
#first line-clearing flush of the day

import time

d.head_check()
b.move_to(flush_pos[0],flush_pos[1])
d.head_descend()
d.single_pump(1000,0,300,100)
d.single_pump(1000,1,300,100)
time.sleep(80)
d.single_pump(1000,0,300,50)
d.single_pump(1000,1,300,50)
time.sleep(60)
d.head_retract()

b.move_to(wick_pos[0],wick_pos[1])
d.head_descend()
time.sleep(10)
d.head_retract()

In [ ]:
d.head_retract()

In [ ]:
# Emergency syringe pump reset if it continues dispensing after an error (or when you make it dispense 0 (ZERO) volume)
d.single_pump(1000,0,10,1)
d.single_pump(1000,1,10,1)

In [ ]:
#Sanity-check concentration calculator
import numpy as np
saltlist = [0.02,0.06,0.1,0.14] #water-salt ratios
salt_conc = 0.5 #Molar solution
salt_conc_list = np.array(saltlist)*salt_conc

print(f"{str(salt_conc_list)} M")

In [ ]:
#Auto-wick
d.head_check() #use stage_webcam_observe
d.head_retract()

b.move_to(wick_pos[0],wick_pos[1])
d.head_descend()
d.head_retract()

In [ ]:
#Shutdown for the day

lamp.off()
d.head_check()
temp.write_sp(10)

#flush line with water
b.move_to(flush_pos[0],flush_pos[1])
d.head_descend()
d.single_pump(1000,0,300,100)
time.sleep(60)
d.head_retract()

b.move_to(wick_pos[0],wick_pos[1])
d.head_descend()
time.sleep(10)
d.head_retract()

In [ ]:
# Background subtraction test (for potential drop footprint calc)

# Setup coordinates/parameters
import numpy as np
import time

lamp = cam.Lamp("Lamp")
lamp.on()

saltlist = [0.00001,0.00001,0.00001,0.00001] #water-salt ratios
salt_conc = 0.5 #Molar solution
salt_conc_list = np.array(saltlist)*salt_conc

print(f"Concentration set: {str(salt_conc_list)} M")

#All relative coordinates to Channel 1 electrode (**Do not remove**)
rel_dep_list = ([0,0,0,0,-15,-15,-15,-15,-30,-30,-30,-30,-45,-45,-45,-45],[0,-15,-30,-45,0,-15,-30,-45,0,-15,-30,-45,0,-15,-30,-45])

ins_1 = [-4,11] #Channel 1 electrode inspection position

vol_tot = 25 # Sample dispense volume
chlist = [1,2] # Select electrodes to use (linked to coordinate positions)

d.head_check()

for i in range(np.size(chlist)):

    vol_salt = vol_tot * saltlist[i]
    vol_water = vol_tot - vol_salt

    # move to empty electrode and snap/store a pic
    b.move_to(ins_1[0]+rel_dep_list[0][chlist[i]-1],ins_1[1]+rel_dep_list[1][chlist[i]-1])

    cam1 = cam.Camera("cam1")
    cam1.open()
    bkgd_pic = cam1.acquire_n_frames(1,0.2) #1 frame with 0.2 s exposure
    cam1.close()
    plt.imshow(bkgd_pic)
    plt.show()

    #flush

    b.move_to(flush_pos[0],flush_pos[1])
    d.head_descend()
    d.single_pump(1000,0,300,vol_water*3) #replace this volume with whatever dead volume we figure out
    d.single_pump(1000,1,300,vol_salt*3) #replace this volume with whatever dead volume we figure out
    time.sleep(60)
    d.head_retract()

    #wick

    b.move_to(wick_pos[0],wick_pos[1])
    time.sleep(2)
    d.head_descend()
    time.sleep(5)
    d.head_retract()

    #deposit
    b.move_to(dep_1[0]+rel_dep_list[0][chlist[i]-1],dep_1[1]+rel_dep_list[1][chlist[i]-1])
    time.sleep(2)
    d.head_descend()
    d.single_pump(1000,0,100,vol_water)
    d.single_pump(1000,1,100,vol_salt)
    time.sleep(30)
    d.head_retract()
    
    #wick again
    b.move_to(wick_pos[0],wick_pos[1])
    d.head_descend()
    time.sleep(5)
    d.head_retract()

    #snap a pic
    b.move_to(ins_1[0]+rel_dep_list[0][chlist[i]-1],ins_1[1]+rel_dep_list[1][chlist[i]-1])

    cam1 = cam.Camera("cam1")
    cam1.open()
    drop_pic = cam1.acquire_n_frames(1,0.2) #1 frame with 0.2 s exposure
    cam1.close()
    plt.imshow(drop_pic)
    plt.show()
        
lamp.off()

# dropcast at that location

# take a pic again

# subtract the two and display the output

In [ ]:
temp.write_sp(60)

In [ ]:
temp.get_pv_surf()

In [ ]:
b.move_to(50,10)

In [76]:
lamp.off()

In [ ]:
d.head_check()
d.head_descend()
d.head_retract()

In [ ]:
# Salt arrayed dropcasting + inspect + EIS run v2. Time-dependent sampling of a single drop in the process of drying over 2 hours.

import numpy as np
import time

lamp = cam.Lamp("Lamp")

saltlist = [0.001,0.1,0.001,0.1,0.002,0.001,0.1,0.003,0.001,0.1] #water-salt ratios
salt_conc = 0.5 #Molar solution

# Universal single-parameter sweep definitions 
exp_var_list = np.array(saltlist)*salt_conc
exp_var = 'NaCl Concentration'
exp_var_unit = 'M'
print(f"{exp_var} set: {str(exp_var_list)} {exp_var_unit}")

vol_tot = 20 # Sample dispense volume

#Electrode #s to use:
chlist = [3,4,5,6,7,8,9,10,11,12] # (linked to coordinate positions)

## Start EIS Settings ##
plotlist = chlist # Select channels to immediately plot in output (if any)
scriptPath = 'SoftAE_classPkg/scripts/testing.mscr'
outPath = 'SoftAE_classPkg/output'
port = EP.palmsens.serial.auto_detect_port() # find the ESPico instrument (only one attached)
p = EP.ESPico('Pico1',port) # name and specify the port of this instance of ESPico class.
## End EIS Settings ## 

d.head_check()

#24 observations 5 minutes apart, for a total of two hours:
timed_observe = 0 #if this is enabled, wait and observe at a single spot
n_iter = 24

if timed_observe == 0:
    n_iter = 1

wait_time = 300 #seconds

for i in range(np.size(chlist)):

    vol_salt = vol_tot * saltlist[i]
    vol_water = vol_tot - vol_salt

    #flush

    b.move_to(flush_pos[0],flush_pos[1])
    d.head_descend()
    d.single_pump(1000,0,300,vol_water*5) #replace this volume with whatever dead volume we figure out
    d.single_pump(1000,1,300,vol_salt*5) #replace this volume with whatever dead volume we figure out
    time.sleep(60)
    d.head_retract()

    #wick

    b.move_to(wick_pos[0],wick_pos[1])
    time.sleep(2)
    d.head_descend()
    time.sleep(5)
    d.head_retract()

    #deposit
    b.move_to(dep_1[0]+rel_dep_list[0][chlist[i]-1],dep_1[1]+rel_dep_list[1][chlist[i]-1])
    time.sleep(2)
    d.head_descend()
    d.single_pump(1000,0,100,vol_water)
    d.single_pump(1000,1,100,vol_salt)
    time.sleep(30)
    d.head_retract()
    
    #wick again
    b.move_to(wick_pos[0],wick_pos[1])
    d.head_descend()
    time.sleep(5)
    d.head_retract()

    for a in range(n_iter):
        lamp.on()

        #snap a pic
        b.move_to(ins_1[0]+rel_dep_list[0][chlist[i]-1],ins_1[1]+rel_dep_list[1][chlist[i]-1])

        cam1 = cam.Camera("cam1")
        cam1.open()
        arr = cam1.acquire_n_frames(1,0.2) #1 frame with 0.2 s exposure
        cam1.close()
        plt.imshow(arr)
        plt.show()

        lamp.off()
        
        # Collect EIS Spectrum

        mscr.eis_run_mscrbuild(scriptPath,chlist[i],10,20000,100,50,0) # write custom EIS MethodScript
        #mscr.lsv_run_mscrbuild(scriptPath,ch) # write custom LSV MethodScript
        c = p.sendscript_getdata(scriptPath,outPath,chlist[i])
        if chlist[i] in plotlist:
            #p.lsv_dataplot(c,chlist[i])
            data = p.eis_dataplot(c,chlist[i])
            print(f'High-freq Z: {str(data[1][0])} Ohms')
            print(f'Lo-freq Z: {str(data[1][-1])} Ohms')
            print(f"{exp_var}: {str(exp_var_list[i])} {exp_var_unit}")
        
        if data[1][0] > 115000: #exceeding conductivity criterion
            break
        elif n_iter == 1:
            break
        else:
            time.sleep(wait_time)
        
lamp.off()

In [ ]:
# Time-dependent observation of a single drop in the process of drying over 2 hours.
# Dropcasting + repeated loop of (inspect + EIS run). 

import numpy as np
import time

lamp = cam.Lamp("Lamp")

saltlist = [0.1] #water-salt ratios
salt_conc = 0.5 #Molar solution
salt_conc_list = np.array(saltlist)*salt_conc

print(f"Concentration set: {str(salt_conc_list)} M")

vol_tot = 60 # Sample dispense volume

## Start EIS Settings ##
chlist = [15] # Select electrodes to use (linked to coordinate positions)
plotlist = chlist # Select channels to immediately plot in output (if any)
scriptPath = 'SoftAE_classPkg/scripts/testing.mscr'
outPath = 'SoftAE_classPkg/output'
port = EP.palmsens.serial.auto_detect_port() # find the ESPico instrument (only one attached)
p = EP.ESPico('Pico1',port) # name and specify the port of this instance of ESPico class.
## End EIS Settings ## 

d.head_check()

#24 observations 5 minutes apart, for a total of two hours:
n_iter = 24
wait_time = 300 #seconds

for i in range(np.size(chlist)):

    # vol_salt = vol_tot * saltlist[i]
    # vol_water = vol_tot - vol_salt

    # #flush

    # b.move_to(flush_pos[0],flush_pos[1])
    # d.head_descend()
    # d.single_pump(1000,0,300,vol_water*3) #replace this volume with whatever dead volume we figure out
    # d.single_pump(1000,1,300,vol_salt*3) #replace this volume with whatever dead volume we figure out
    # time.sleep(60)
    # d.head_retract()

    # #wick

    # b.move_to(wick_pos[0],wick_pos[1])
    # time.sleep(2)
    # d.head_descend()
    # time.sleep(5)
    # d.head_retract()

    # #deposit
    # b.move_to(dep_1[0]+rel_dep_list[0][chlist[i]-1],dep_1[1]+rel_dep_list[1][chlist[i]-1])
    # time.sleep(2)
    # d.head_descend()
    # d.single_pump(1000,0,100,vol_water)
    # d.single_pump(1000,1,100,vol_salt)
    # time.sleep(30)
    # d.head_retract()
    
    # #wick again
    # b.move_to(wick_pos[0],wick_pos[1])
    # d.head_descend()
    # time.sleep(5)
    # d.head_retract()

    for a in range(n_iter):
        i = 0
        lamp.on()

        #snap a pic
        b.move_to(ins_1[0]+rel_dep_list[0][chlist[i]-1],ins_1[1]+rel_dep_list[1][chlist[i]-1])

        cam1 = cam.Camera("cam1")
        cam1.open()
        arr = cam1.acquire_n_frames(1,0.2) #1 frame with 0.2 s exposure
        cam1.close()
        plt.imshow(arr)
        plt.show()

        lamp.off()
        
        # Collect EIS Spectrum

        mscr.eis_run_mscrbuild(scriptPath,chlist[i],10,20000,100,50,0) # write custom EIS MethodScript
        #mscr.lsv_run_mscrbuild(scriptPath,ch) # write custom LSV MethodScript
        c = p.sendscript_getdata(scriptPath,outPath,chlist[i])
        if chlist[i] in plotlist:
            #p.lsv_dataplot(c,chlist[i])
            data = p.eis_dataplot(c,chlist[i])
            print(f'High-freq Z: {str(data[1][0])} Ohms')
            print(f'Lo-freq Z: {str(data[1][-1])} Ohms')
        
        if data[1][0] > 150000: #exceeding conductivity criterion
            break
        else:
            time.sleep(wait_time)
        
lamp.off()

In [ ]:
#Acquire images

chlist = [5,6,7,8]

d.head_check()
lamp.on()

for i in range(np.size(chlist)):

    b.move_to(ins_1[0]+rel_dep_list[0][chlist[i]-1],ins_1[1]+rel_dep_list[1][chlist[i]-1])

    cam1 = cam.Camera("cam1")
    cam1.open()
    arr = cam1.acquire_n_frames(1,0.2) #1 frame with 0.2 s exposure
    cam1.close()
    plt.imshow(arr)
    plt.show()

lamp.off()

In [ ]:
#Testing conductivity on each channel with calibration solutions. 1-8 is 45 uS/cm, 9-16 is 4500 uS/cm.
# From a first passs looks like absolute measurements vary with drop location- and/or drop size, but remain in the same ballpark.

port = EP.palmsens.serial.auto_detect_port() # find the ESPico instrument (only one attached)
p = EP.ESPico('Pico1',port) # name and specify the port of this instance of ESPico class.
chlist = range(1,17) # Define multiplexer channel list
# if you want to select all channels within a range use the below: 

#for i in range(1,6): #range is exclusive of end index: range(m,n) is m to n-1
#    chlist.append(i)

plotlist = range(1,17) #if you want to selectively see the data

scriptPath = 'SoftAE_classPkg/scripts/testing.mscr'
outPath = 'SoftAE_classPkg/output'

# multichannel data recall and selective plotting
for ch in chlist:
    mscr.eis_run_mscrbuild(scriptPath,ch,10,20000,100,50,0) # write custom EIS MethodScript
    #mscr.lsv_run_mscrbuild(scriptPath,ch) # write custom LSV MethodScript
    c = p.sendscript_getdata(scriptPath,outPath,ch)
    if ch in plotlist:
        #p.lsv_dataplot(c,ch)
        data = p.eis_dataplot(c,ch)
        print(f'High-freq Z: {str(data[1][0])} Ohms')

In [ ]:
import time
#b.stage_init()
#b.home_stage()
#Testing pcb location 1

#flush_pos = [-34.5,5]
ins_1 = [-4.5,9.5]
dep_1 = [45,50]

ins_offset_2x = [-49,-39]

b.move_to(dep_1[0],dep_1[1]) #electrode 1

rel_dep_list = ([0,0,0,0,-15,-15,-15,-15,-30,-30,-30,-30,-45,-45,-45,-45],[0,-15,-30,-45,0,-15,-30,-45,0,-15,-30,-45,0,-15,-30,-45])


#b.move_by(28, -3) #rough calibration - either keep constant or always check
#28 mm in motor 1 direction positive from microscope to syring
#10 mm in motor 2 direction negative from microscope to syringe
#therefore do move_by(28, -10)



In [ ]:
# Simple temperature ramp script (holds everything else up while running) -- move to tempControl class
# (need to multi-thread, or insert tasks inside as nested loops if we want other things to run concurrently.)
from SoftAE_classPkg import tempControl_class as t

temp = t.tempControl("stage","com6",115200,0,1) #Novus instance

temp.ramp_linear(43,38,1200,50)
#temp.write_sp(44)

In [ ]:
temp.get_pv_surf()

In [8]:
# Test cycle laser diode for stage alignment
import nidaqmx
import time

with nidaqmx.Task() as task:
    task.ao_channels.add_ao_voltage_chan("Dev1/ao0","stageCalDiode",0,5)
    v_list = [0.5,3.7,4.5,5,4.2,3.7,0]
    for i in v_list:
        task.write(i,auto_start=True)
        time.sleep(0.5)

In [ ]:
# Integration example 2:Incorporate optimizer into 16-point array
import numpy as np
import time
bigollist = ([1, 0.5, 0.1, 0])
saltlist = ([0, 0.5, 0.9, 1.0]) #array of 0 - water (first), 1 - salt
move_arr = b.array_pos([-50,-24.9, 3.2,30.1],[21,31,31,21])
#b.move_to(-50, 21) #electrode 5
b.move_to(0,-40) #Move to arbitrary flush position
d.single_pump(10, 0, 100, 50)
time.sleep(60) #Consider the merits of flushing between runs pls


for i in range(len(move_arr[1,:])):

b.move_to(0,-40)
d.single_pump(10, 0, 100, 50)
time.sleep(30)
port = EP.palmsens.serial.auto_detect_port() # find the ESPico instrument (only one attached)
p = EP.ESPico('Pico1',port) # name and specify the port of this instance of ESPico class.
chlist = [5,6,7,8] # Define multiplexer channel list
# if you want to select all channels within a range use the below: 

#for i in range(1,6): #range is exclusive of end index: range(m,n) is m to n-1
#    chlist.append(i)

plotlist = [5,6,7,8] #if you want to selectively see the data

scriptPath = 'SoftAE_classPkg/scripts/testing.mscr'
outPath = 'SoftAE_classPkg/output'

# multichannel data recall and selective plotting
for ch in chlist:
    mscr.eis_run_mscrbuild(scriptPath,ch,10,20000,100,50,0) # write custom EIS MethodScript
    #mscr.lsv_run_mscrbuild(scriptPath,ch) # write custom LSV MethodScript
    c = p.sendscript_getdata(scriptPath,outPath,ch)
    if ch in plotlist:
        #p.lsv_dataplot(c,ch)
        p.eis_dataplot(c,ch)

In [ ]:
# Integration example 1: testing PCB array motion, inspection, drop-casting, and EIS measurement.
import numpy as np
import time
bigollist = ([1, 0.5, 0.1, 0])
saltlist = ([0, 0.5, 0.9, 1.0]) #array of 0 - water (first), 1 - salt
move_arr = b.array_pos([-50,-24.9, 3.2,30.1],[21,31,31,21])
#b.move_to(-50, 21) #electrode 5
b.move_to(0,-40) #Move to arbitrary flush position
d.single_pump(10, 0, 100, 50)
time.sleep(60) #Consider the merits of flushing between runs pls


for i in range(len(move_arr[1,:])):
    b.move_to(move_arr[0,i],move_arr[1,i])
    cam1 = cam.Camera("cam1")
    b.move_by(14,-25) #rough calibration - either keep constant or always check 
    #11/18 - moves to check location, then moves to dispense onto electrode. Flip order.
    #b.move_by(-14, 25)
    #Move_arr needs to be move_arr+14 in x, -25 in y to go from camera to syringe.
    d.single_pump(10, 0, 100, 20*bigollist[i])
    d.single_pump(10, 1, 100, 20*saltlist[i])
    time.sleep(60)

    # cam1 = cam.Camera("cam1")
    # cam1.open()
    # arr = cam1.acquire_n_frames(1,0.5) #1 frame with 0.5 s exposure
    # cam1.close()
    # plt.imshow(arr)
    # plt.show()
b.move_to(0,-40)
d.single_pump(10, 0, 100, 50)
time.sleep(30)
port = EP.palmsens.serial.auto_detect_port() # find the ESPico instrument (only one attached)
p = EP.ESPico('Pico1',port) # name and specify the port of this instance of ESPico class.
chlist = [5,6,7,8] # Define multiplexer channel list
# if you want to select all channels within a range use the below: 

#for i in range(1,6): #range is exclusive of end index: range(m,n) is m to n-1
#    chlist.append(i)

plotlist = [5,6,7,8] #if you want to selectively see the data

scriptPath = 'SoftAE_classPkg/scripts/testing.mscr'
outPath = 'SoftAE_classPkg/output'

# multichannel data recall and selective plotting
for ch in chlist:
    mscr.eis_run_mscrbuild(scriptPath,ch,10,20000,100,50,0) # write custom EIS MethodScript
    #mscr.lsv_run_mscrbuild(scriptPath,ch) # write custom LSV MethodScript
    c = p.sendscript_getdata(scriptPath,outPath,ch)
    if ch in plotlist:
        #p.lsv_dataplot(c,ch)
        p.eis_dataplot(c,ch)